### Attempting to Find Patterns: 

- As it stands my current solution won't work as it takes too long to implement. 
- Will need to identify a pattern & leverage this to solve properly

- Going to test on this: `12345678`

In [1]:
from itertools import cycle, islice
import numpy as np 
def rebuild_base(row_num, base_pattern, num_rows):

   
    # multiply base by num row
    updated_base = [[x] * row_num for x in base_pattern]
   
    # flatten into one list - should maintain order
    flattened =  [item for sublist in updated_base for item in sublist]
   
    # make it large enough for length of input: we add one due to the shift
    long_flat = list(islice(cycle(flattened), num_rows +1))
   
    # shift by 1
    new_base = long_flat[1:]
    
    # limit by leading 0s: due to offset we have to subtract 1
    new_base = new_base[row_num-1:]
   
    return tuple(new_base)

In [2]:
rebuild_base(1, base_pattern, 8)

NameError: name 'base_pattern' is not defined

In [ ]:
# start in phase 1
phase = 1

# our base
base_pattern = [0, 1, 0, -1]

# starting signal:
input_signal = '12345678'

In [122]:
%timeit 3

phase_dict = {}

while phase < 101:
    
    
    row_dict = {}
    
    input_list = []

    # iterate over & generate proper list
    for i in input_signal:

        input_list.append(int(i))
   
    # get number of rows needed
    num_rows = len(input_list)
    
    # convert to array for speed
    input_list = np.asarray(input_list)
   
    # temp dict to store output
    temp_dict = {}
    
    for row_num in range(1,num_rows+1):

        # update base according to row num
        new_base = rebuild_base(row_num, base_pattern, num_rows)

        # zip with input list
        row_sum = 0
        row_list = []
        
        # all up to row_num -1 are 0. if row 1, then 1 leading 0; if row 2, then 2 leading 0; etc. 
        for x,y in zip(new_base, input_list[row_num-1:]):

            row_sum += x*y
            row_list.append((x,y))

        # add sum to row dict
        row_dict[row_num] = str(abs(row_sum))[-1]
        
        # update: I want to see the pattern, so storing as a new dict
        temp_dict[row_num] = (row_list, str(abs(row_sum))[-1])
        print(f"{temp_dict[row_num]}")
        
    input_signal = ''.join([val for val in row_dict.values()])
    phase_dict[phase] = temp_dict
    
    print(f'Phase {phase} output signal first 8 digits is: {input_signal} ')
    phase +=1 # increment by 1


11.1 ns ± 0.557 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


### Part 2: 

- Cleaned the previous code up slightly....also need to apply the offset....

- Moving from Phase 5 to Phase 6 we see this...

```
Phase 5 output signal first 8 digits is: 02140178 
([(1, 0), (0, 2), (-1, 1), (0, 4), (1, 0), (0, 1), (-1, 7), (0, 8)], '8')
([(1, 2), (1, 1), (0, 4), (0, 0), (-1, 1), (-1, 7), (0, 8)], '5')
([(1, 1), (1, 4), (1, 0), (0, 1), (0, 7), (0, 8)], '5')
([(1, 4), (1, 0), (1, 1), (1, 7), (0, 8)], '2')
([(1, 0), (1, 1), (1, 7), (1, 8)], '6')
([(1, 1), (1, 7), (1, 8)], '6')
([(1, 7), (1, 8)], '5')
([(1, 8)], '8')
```

Bottom: 

- 8th row output = 8th  input (8 -> 8)
- 7th row output = 8th input + 7th  input (15 -> 5)
- 6th row output = 8th  + 7th  + 6th  input (16) -> 6
- 5th row output = 8th  + 7th  + 6th  + 5th input (16) -> 6


So, it seems like we can solve the bottom half very quickly? 

In [64]:
num_rows = 8
import math

In [71]:
for row_num in range(1,math.ceil((num_rows+1)/2)):
    print(row_nu)

1
2
3
4


In [72]:
for row_num in reversed(range(math.ceil((num_rows+1)/2), num_rows+1)):
    print(row_num)

8
7
6
5


In [121]:
%timeit 3

# start in phase 1
phase = 1

# our base
base_pattern = [0, 1, 0, -1]

# starting signal:
input_signal = '12345678'

phase_dict = {}

while phase < 101:
    
    
    row_dict = {}
    
    input_list = []

    # iterate over & generate proper list
    for i in input_signal:

        input_list.append(int(i))
   
    # get number of rows needed
    num_rows = len(input_list)
    
    # convert to array for speed
    input_list = np.asarray(input_list)
    
    for row_num in range(1,math.ceil((num_rows+1)/2)):

        # update base according to row num
        new_base = rebuild_base(row_num, base_pattern, num_rows)

        # zip with input list
        row_sum = 0
        
        # all up to row_num -1 are 0. if row 1, then 1 leading 0; if row 2, then 2 leading 0; etc. 
        for x,y in zip(new_base, input_list[row_num-1:]):

            row_sum += x*y
            row_list.append((x,y))

        # add sum to row dict
        row_dict[row_num] = str(abs(row_sum))[-1]
        
    for row_num in range(math.ceil((num_rows+1)/2), num_rows+1):

        # second half we can just sum the values in list up to that point
        row_dict[row_num] = str(abs(sum(input_list[row_num-1:])))[-1]
        
        
    input_signal = ''.join([val for val in row_dict.values()])
    phase_dict[phase] = temp_dict
    
    print(f'Phase {phase} output signal first 8 digits is: {input_signal} ')
    phase +=1 # increment by 1


11.2 ns ± 0.987 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
Phase 1 output signal first 8 digits is: 48226158 
Phase 2 output signal first 8 digits is: 34040438 
Phase 3 output signal first 8 digits is: 03415518 
Phase 4 output signal first 8 digits is: 01029498 
Phase 5 output signal first 8 digits is: 02140178 
Phase 6 output signal first 8 digits is: 85526658 
Phase 7 output signal first 8 digits is: 41395938 
Phase 8 output signal first 8 digits is: 38765018 
Phase 9 output signal first 8 digits is: 04824998 
Phase 10 output signal first 8 digits is: 36440678 
Phase 11 output signal first 8 digits is: 83871158 
Phase 12 output signal first 8 digits is: 45645438 
Phase 13 output signal first 8 digits is: 04560518 
Phase 14 output signal first 8 digits is: 63124498 
Phase 15 output signal first 8 digits is: 09795178 
Phase 16 output signal first 8 digits is: 98121658 
Phase 17 output signal first 8 digits is: 42440938 
Phase 18 output signal first 8 digits is: 3686

#### Update to 1000 

In [51]:
# start in phase 1
phase = 1

# our base
base_pattern = [0, 1, 0, -1]

# starting signal:
input_signal = '03036732577212944063491565474664'

# generate signal copied 1000 times
big_list = input_signal * 10000

In [52]:
# pull out the tail end
v = big_list[int(input_signal[:7]):] 

In [53]:
len(big_list)

320000

In [54]:
input_signal = v

In [55]:
import math

In [50]:
temp_dict = {}
phase_dict = {}

while phase < 101:
    
    
    row_dict = {}
    
    input_list = []

    # iterate over & generate proper list
    for i in input_signal:

        input_list.append(int(i))
   
    # get number of rows needed
    num_rows = len(input_list)
    
    # convert to array for speed
    input_list = np.asarray(input_list)
        
    for row_num in reversed(range(1, num_rows+1)):

        # second half we can just sum the values in list up to that point
        row_dict[row_num] = str(abs(sum(input_list[row_num-1:])))[-1]
        
        
    input_signal = ''.join([val for val in row_dict.values()])
    phase_dict[phase] = temp_dict
    
    print(f'Phase {phase} output signal first 8 digits is: {input_signal} ')
    phase +=1 # increment by 1

Phase 1 output signal first 8 digits is: 61706040077481861475423955962327162515955229363169209784004178726170604007748186147542395596232716251595522936316920978400417872617060400774818614754239559623271625159552293631692097840041787261706040077481861475423955962327162515955229363169209784004178726170604007748186147542395596232716251595522936316920978400417872617060400774818614754239559623271625159552293631692097840041787261706040077481861475423955962327162515955229363169209784004178726170604007748186147542395596232716251595522936316920978400417872617060400774818614754239559623271625159552293631692097840041787261706040077481861475423955962327162515955229363169209784004178726170604007748186147542395596232716251595522936316920978400417872617060400774818614754239559623271625159552293631692097840041787261706040077481861475423955962327162515955229363169209784004178726170604007748186147542395596232716251595522936316920978400417872617060400774818614754239559623271625159552293631692097840041787

Phase 2 output signal first 8 digits is: 48700440003624359847286349459742548610561642304104533479555103586092266222584657106940856167196476083278386452632675569177732570821448844470687932816207838931869820549050867485489778139995479204366006669280915403842905015308104276127208960760199035111769142658822888140213762506412723752032649834942018298231125733398136487004400036243598472863494597425486105616423041045334795551035860922662225846571069408561671964760832783864526326755691777325708214488444706879328162078389318698205490508674854897781399954792043660066692809154038429050153081042761272089607601990351117691426588228881402137625064127237520326498349420182982311257333981364870044000362435984728634945974254861056164230410453347955510358609226622258465710694085616719647608327838645263267556917773257082144884447068793281620783893186982054905086748548977813999547920436600666928091540384290501530810427612720896076019903511176914265882288814021376250641272375203264983494201829823112573339813

KeyboardInterrupt: 